In [5]:
# %matplotlib notebook
%matplotlib widget
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Quickstart

In this tutorial we show the basic usage of the reloaded Rossiter-McLaughlin model for the stellar surface velocity as implemented in `elle`. You will need `numpy`, `matplotlib`, `elle`, and `emcee` to run this tutorial.

We will start by defining the orbital parameters of a typical hot Jupiter system, and create some synthetic stellar surface velocity data that we may have retrieved from an analysis of the in-transit cross-correlation functions (CCFs). For an in-depth guide of how to derive the stellar surface velocity data from CCFs, see the tutorial for HD189733b in `<name>`.

In [94]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("/Users/vxh710/PhD/software/elle")
import utils, model

period = 3                              # orbital period (days)
t0     = 0                              # transit time (BJD)
roa    = 0.1                            # scaled stellar radius
ror    = 0.1                            # planet-star radius ratio
b      = 0.5                            # impact parameter
incl   = np.rad2deg(np.arccos(b * roa)) # orbital inclination (deg)
u      = [0.4, 0.3]                     # limb darkening coefficients for quadratic law
vsini  = 3                              # stellar projected rotation (km/s)
ell    = 0                              # spin-orbit angle (deg)

# compute transit duration between contacts
dur14 = utils.get_14_transit_duration(period, roa, ror, b, np.deg2rad(incl))
dur23 = utils.get_23_transit_duration(period, roa, ror, b, np.deg2rad(incl))

# timestamps with 10min exposures and 1min dead-time
phase = np.arange(t0 - 0.45 * dur14,
                  t0 + 0.45 * dur14,
                  (10+1) / (60*24)
                ) / period

istar = 90. # stellar inclination
alpha = 0. # assume rigid body, no differential rotation

# create the orbit model that describes the transit chord along the surface of the star
model_kwargs = {'r_1':roa, 'i_p':incl, 'r_p':ror, 'ld':'quad', 'ldc':u, 'Nxy':51}
orbit = model.ReloadedModel(phase, **model_kwargs)

# get the radial velocity of the local stellar surface (at each transit epoch)
rv  = orbit(vsini, ell, istar, alpha)

# generate data with some noise
np.random.seed(17)
N = len(rv)
rv_err = 0.1 + 0.02 * np.random.randn(N)
rv += rv_err * np.random.randn(N)



Note that when using `elle`, we first define an `orbit` which describes the path of the transit chord across the stellar disc given some orbital parameters. This model is then used to compute the surface velocity at each epoch based on several stellar parameters that define the star's rotation and orientation relative to the planet orbit.

In the plot below we show our synthetic data along with some lines that denote the transit contacts.

In [90]:
plt.figure()

# transit contacts
c1, c2, c3, c4 = 0.5 / period * np.array([-dur14, -dur23, dur23, dur14])

plt.axvline(c1, c='#aaaaaa', lw=1, ls='solid')
plt.axvline(c2, c='#aaaaaa', lw=1, ls='dotted')
plt.axvline(c3, c='#aaaaaa', lw=1, ls='dotted')
plt.axvline(c4, c='#aaaaaa', lw=1, ls='solid')

plt.errorbar(phase, rv, rv_err, capsize=0, c='k', fmt='.', elinewidth=0.5)
plt.xlabel('phase')
plt.ylabel('stellar surface velocity (km/s)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'stellar surface velocity (km/s)')

Looks good! We will fit these data using `emcee`, where our simplest model only consists of two parameters: the projected stellar rotation $v\sin{i}$, and sky-projected spin-orbit angle $\lambda$. We will instead sample two related parameters, $\sqrt{v\sin{i}} \sin{\lambda}$ and $\sqrt{v\sin{i}} \cos{\lambda}$ (e.g. Triaud+ XXXX), which can sometimes improve performance.

In [91]:
from multiprocessing import Pool
import emcee
sys.path.append("/Users/vxh710/PhD/software/elle")
import model

bounds = (-10, 10) # uniform bounds on sqrt(vsini) * {sin lambda, cos lambda}

def _log_prior(theta):
        
    vs, vc = theta
    
    if vs < bounds[0] or vs > bounds[1]:
        return -np.inf
    elif vc < bounds[0] or vc > bounds[1]:
        return -np.inf

    return 0

def _log_likelihood(data, model, error):
        inv_sigma2 = 1/error**2
        return -0.5 * np.sum((data - model)**2 * inv_sigma2 - np.log(inv_sigma2))

def _log_probability(theta):

    # calculate prior and check the new parameters are within bounds
    l = _log_prior(theta)
    
    if not np.isfinite(l):
        return -np.inf
    
    # calculate vsini and lambda from the free parameters
    vs, vc = theta # vs = sqrt(vsini) * np.sin(lambda); vc = sqrt(vsini) * np.cos(lambda)
    vsini = vs**2 + vc**2  
    ell = np.rad2deg(np.arctan2(vs, vc))

    mod = orbit(vsini, ell, istar, alpha) # calculate surface RV model

    l += _log_likelihood(rv, mod, rv_err)
    
    return l

parameters = ['vs', 'vc']
ndim = len(parameters)

threads = 4
walkers = 200
steps = 10000

init = np.random.uniform(*bounds, (walkers, 2))
    
if threads > 1:
    os.environ["OMP_NUM_THREADS"] = "1"
    with Pool(processes=threads) as pool:
        sampler = emcee.EnsembleSampler(walkers, ndim,
                                        _log_probability,
                                        pool=pool)
        sampler.run_mcmc(init, steps, progress=True)
else:
    sampler = emcee.EnsembleSampler(walkers, ndim,
                                        _log_probability)
    sampler.run_mcmc(init, steps, progress=True)

100%|██████████| 10000/10000 [13:44<00:00, 12.12it/s]


In [92]:
# discard = int(0.5 * steps)
# thin = int(np.mean(sampler.get_autocorr_time(discard=discard)))

# stepsarr = np.arange(int((steps-discard)/thin))

# fig, axes = plt.subplots(ndim+1,1, figsize=(10,2*ndim),
#         gridspec_kw={"hspace":0.04})


# vsini = np.sum(sampler.get_chain(discard=discard, thin=thin)**2, axis=-1)
# ell = np.rad2deg(np.arctan2(*np.rollaxis(sampler.get_chain(discard=discard, thin=thin), 2, 0)))
# posterior_3d = np.dstack((vsini, ell))


# labels = ['logp', '$v\sin{i}$ (km/s)', '$\lambda$ (deg)']

# for i in range(ndim+1):
#     axes[i].set_xlim(0, stepsarr.max())
#     for j in range(walkers):
#         if i == 0:
#             axes[i].plot(stepsarr, sampler.get_log_prob(discard=discard, thin=thin)[:,j], lw=0.5)
#         else:
#             axes[i].plot(stepsarr, posterior_3d[:,j,i-1], lw=0.5)
#         axes[i].set_ylabel(labels[i])
#         if i == ndim:
#             axes[i].set_xlabel('steps')
#         else:
#             axes[i].tick_params(labelbottom=False)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [95]:
import corner

# discard burn-in points and thin chains by the autocorrelation length
discard = int(0.5 * steps)
thin = int(np.mean(sampler.get_autocorr_time(discard=discard)))

# compute vsini and lambda
posterior = sampler.get_chain(discard=discard, thin=thin, flat=True)
_vsini = np.sum(posterior**2, axis=-1)
_ell = np.rad2deg(np.arctan2(*posterior.T))
posterior = np.column_stack((_vsini, _ell))

labels = ["$v\sin{i}$ (km/s)", "$\lambda$ (deg)"]

fig = corner.corner(posterior,
             labels=labels, show_titles=True, title_fmt=".2f",
            truths=[vsini, ell])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

From the corner plot above we see that we recover the true values. Finally, we plot the best-fitting model to the data.

In [97]:
theta_ml = posterior[np.argmax(sampler.get_log_prob(discard=discard, thin=thin, flat=True))]

rv_mod = orbit(*theta_ml, istar, alpha, phase=phase)

phase_f = np.linspace(-0.5 * dur14/period, 0.5 * dur14/period, 200)
rv_mod_f = orbit(*theta_ml, istar, alpha, phase=phase_f)

gridspec_kw = {
              'height_ratios':[3,1], 
              'hspace':0.03,
              'wspace':0.02
              }

fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw=gridspec_kw)


ax1.set_ylabel('stellar surface velocity (km/s)')
ax1.tick_params(axis='x', which='both', labelbottom=False)
ax2.set_xlabel('phase')
ax2.set_ylabel("O - C (km/s)")

c1, c2, c3, c4 = 0.5 / period * np.array([-dur14, -dur23, dur23, dur14])

ax1.axvline(c1, c='#aaaaaa', lw=1, ls='solid')
ax1.axvline(c2, c='#aaaaaa', lw=1, ls='dotted')
ax1.axvline(c3, c='#aaaaaa', lw=1, ls='dotted')
ax1.axvline(c4, c='#aaaaaa', lw=1, ls='solid')

ax1.errorbar(phase, rv, yerr=rv_err, capsize=0, c='k', fmt='.', elinewidth=0.5)
ax1.plot(phase_f, rv_mod_f, color='C0', lw=1.5)

ax2.errorbar(phase, rv - rv_mod, yerr=rv_err, capsize=0, c='k', fmt='.', elinewidth=0.5)
ax2.axhline(0, c="#aaaaaa", lw=1.5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …